In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import random
import scipy.signal
from skimage.restoration import denoise_tv_bregman
from skimage import restoration
import skimage
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

In [12]:
random.seed(200)

tf.keras.backend.clear_session()
tf.config.optimizer.set_jit(False) # Start with XLA disabled.

def load_data():
  fashion_mnist = tf.keras.datasets.fashion_mnist

  (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()


  x_train = x_train.astype('float32') / 256
  x_test = x_test.astype('float32') / 256

  # Convert class vectors to binary class matrices.
  y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
  y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)
  return ((x_train, y_train), (x_test, y_test))

(x_train, y_train), (x_test, y_test) = load_data()

img = Image.new('L', (28, 28), 255)
draw = ImageDraw.Draw(img)
for y in range(5, 91, 4):
    draw.line((32, y, 0, y), (0), 2)

def struct_noise(image):
   im = Image.fromarray((image * 255).astype(np.uint8)) 
   noisyimage = Image.blend(im, img, 0.5)
   return noisyimage




#test 1: train0 test0 = base splits

#test 2: train0 test75
a,b,an,bn = train_test_split(x_test, y_test, test_size=0.25, random_state=44)
a0 = np.empty_like(a)
y2 = np.empty_like(an)
ytv2 = np.empty_like(an)
atv = np.empty_like(a)
for j in range(len(a)):
    a0[j] = struct_noise(a[j])
    atv[j] = skimage.restoration.denoise_tv_bregman(a0[j])
    y2[j] = an[j]
    ytv2[j] = an[j]
a0 = np.concatenate((a0,b))
atv = np.concatenate((atv,b))
y2 = np.concatenate((y2,bn))
ytv2 = np.concatenate((ytv2, bn))

#test 3: train0 test50
c,d,cn,dn = train_test_split(x_test, y_test, test_size=0.50, random_state=44)
c0 = np.empty_like(c)
ctv = np.empty_like(c)
y3 = np.empty_like(cn)
ytv3 = np.empty_like(cn)
for j in range(len(c)):
    c0[j] = struct_noise(c[j])
    ctv[j] = skimage.restoration.denoise_tv_bregman(c0[j])
    y3[j] = cn[j]
    ytv3[j] = cn[j]
c0 = np.concatenate((c0,d))
ctv = np.concatenate((ctv,d))
y3 = np.concatenate((y3,dn))
ytv3 = np.concatenate((ytv3, dn))

#test 4: train0 test25
e,f,en,fn = train_test_split(x_test,y_test, test_size=0.75, random_state=44)
e0 = np.empty_like(e)
etv = np.empty_like(e)
y4 = np.empty_like(en)
ytv4 = np.empty_like(en)
for j in range(len(e)):
    e0[j] = struct_noise(e[j])
    etv[j] = skimage.restoration.denoise_tv_bregman(e0[j])
    y4[j] = en[j]
    ytv4[j] = en[j]
e0 = np.concatenate((e0,f))
etv = np.concatenate((etv,f))
y4 = np.concatenate((y4,fn))
ytv4 = np.concatenate((ytv4, fn))

#test 5: train0 test100
g0 = np.empty_like(x_test)
gtv = np.empty_like(x_test)
for j in range(len(x_test)):
    g0[j] = struct_noise(x_test[j])
    gtv[j] = skimage.restoration.denoise_tv_bregman(g0[j])

In [14]:
def generate_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', input_shape=(28,28,1)),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Conv2D(32, (3, 3)),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(64, (3, 3), padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Conv2D(64, (3, 3)),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Activation('softmax')
  ])

model = generate_model()

In [15]:
def compile_model(model):
  opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
  model.compile(loss='categorical_crossentropy',
                optimizer=opt,
                metrics=['accuracy'])
  return model

model = compile_model(model)

def train_model(model, x_train, y_train, x_test, y_test, epochs=25):
  model.fit(x_train, y_train, batch_size=256, epochs=epochs, validation_data=(x_test, y_test), shuffle=True)

train_model(model, x_train,y_train, x_test,y_test)

Epoch 1/25
235/235 [==============================] - 27s 111ms/step - loss: 1.1423 - accuracy: 0.5824 - val_loss: 0.6405 - val_accuracy: 0.7601
Epoch 2/25
235/235 [==============================] - 31s 131ms/step - loss: 0.6332 - accuracy: 0.7630 - val_loss: 0.5266 - val_accuracy: 0.8015
Epoch 3/25
235/235 [==============================] - 29s 124ms/step - loss: 0.5440 - accuracy: 0.8004 - val_loss: 0.4729 - val_accuracy: 0.8235
Epoch 4/25
235/235 [==============================] - 29s 122ms/step - loss: 0.4980 - accuracy: 0.8167 - val_loss: 0.4387 - val_accuracy: 0.8440
Epoch 5/25
235/235 [==============================] - 28s 121ms/step - loss: 0.4663 - accuracy: 0.8303 - val_loss: 0.4108 - val_accuracy: 0.8519
Epoch 6/25
235/235 [==============================] - 29s 122ms/step - loss: 0.4369 - accuracy: 0.8397 - val_loss: 0.3951 - val_accuracy: 0.8576
Epoch 7/25
235/235 [==============================] - 28s 121ms/step - loss: 0.4146 - accuracy: 0.8483 - val_loss: 0.3759 - val_ac

In [16]:
scores_t1 = model.evaluate(x_test, y_test, verbose=1)
print('Benchmark Test loss:', scores_t1[0])
print('Benchmark Test accuracy:', scores_t1[1])

base_scores_t2 = model.evaluate(a0, y2, verbose=1)
print('B75 Test loss:', base_scores_t2[0])
print('B75 Test accuracy:', base_scores_t2[1])

tv_scores_t2 = model.evaluate(atv, ytv2, verbose=1)
print('T75 Test loss:', tv_scores_t2[0])
print('T75 Test accuracy:', tv_scores_t2[1])

base_scores_t3 = model.evaluate(c0, y3, verbose=1)
print('B50 Test loss:', base_scores_t3[0])
print('B50 Test accuracy:', base_scores_t3[1])

tv_scores_t3 = model.evaluate(ctv, ytv3, verbose=1)
print('T50 Test loss:', tv_scores_t3[0])
print('T50 Test accuracy:', tv_scores_t3[1])

base_scores_t4 = model.evaluate(e0, y4, verbose=1)
print('B25 Test loss:', base_scores_t4[0])
print('B25 Test accuracy:', base_scores_t4[1])

tv_scores_t4 = model.evaluate(etv, ytv4, verbose=1)
print('T25 Test loss:', tv_scores_t4[0])
print('T25 Test accuracy:', tv_scores_t4[1])

base_scores_t5 = model.evaluate(g0, y_test, verbose=1)
print('B100 Test loss:', base_scores_t5[0])
print('B100 Test accuracy:', base_scores_t5[1])

tv_scores_t5 = model.evaluate(gtv, y_test, verbose=1)
print('T100 Test loss:', tv_scores_t5[0])
print('T100 Test accuracy:', tv_scores_t5[1])

313/313 [==============================] - 2s 5ms/step - loss: 0.2516 - accuracy: 0.9077
Benchmark Test loss: 0.25160714983940125
Benchmark Test accuracy: 0.9077000021934509
313/313 [==============================] - 2s 5ms/step - loss: 25.7919 - accuracy: 0.8243
B75 Test loss: 25.791934967041016
B75 Test accuracy: 0.8242999911308289
313/313 [==============================] - 2s 5ms/step - loss: 25.6284 - accuracy: 0.8233
T75 Test loss: 25.628435134887695
T75 Test accuracy: 0.8233000040054321
313/313 [==============================] - 2s 5ms/step - loss: 17.3284 - accuracy: 0.8529
B50 Test loss: 17.328393936157227
B50 Test accuracy: 0.8529000282287598
313/313 [==============================] - 2s 5ms/step - loss: 17.2157 - accuracy: 0.8522
T50 Test loss: 17.21567726135254
T50 Test accuracy: 0.8521999716758728
313/313 [==============================] - 2s 6ms/step - loss: 8.9008 - accuracy: 0.8801
B25 Test loss: 8.90079116821289
B25 Test accuracy: 0.8801000118255615
313/313 [===========